# Data Anonymization

In Pega CDH 8.5 and up, it's now possible to record the historical data as seen by the Adaptive Models. See [this academy challenge](https://academy.pega.com/challenge/exporting-historical-data/v4) for reference. This historical data can be further used to experiment with offline models, but also to fine-tune the OOTB Gradient Boosting model. However, sharing this information with Pega can be sensitive as it contains raw predictor data. 

To this end, we provide a simple and transparent script to fully anonimize this dataset.

The DataAnonymization script is now part of pdstools, and you can import it directly as such.

In [1]:
# These lines are only for rendering in the docs, and are hidden through Jupyter tags
# Do not run if you're running the notebook seperately

import os  
import sys
import plotly.io as pio
pio.renderers.default = "notebook_connected"

sys.path.append("../../../")
sys.path.append('../../python')

In [2]:
from pdstools import ADMDatamart
from pdstools import Config, DataAnonymization
import polars as pl

## Input data

To demonstrate this process, we're going to anonymise this toy example dataframe:

In [3]:
pl.read_ndjson('../../../../data/SampleHDS.json')

Context_Name,Customer_MaritalStatus,Customer_CLV,Customer_City,IH_Web_Inbound_Accepted_pxLastGroupID,Decision_Outcome
str,str,i64,str,str,str
"""FirstMortgage30yr""","""Married""",1460,"""Port Raoul""","""Account""","""Rejected"""
"""FirstMortgage30yr""","""Unknown""",669,"""Laurianneshire""","""AutoLoans""","""Accepted"""
"""MoneyMarketSavingsAccount""","""No Resp+""",1174,"""Jacobshaven""","""Account""","""Rejected"""
"""BasicChecking""","""Unknown""",1476,"""Lindton""","""Account""","""Rejected"""
"""BasicChecking""","""Married""",1211,"""South Jimmieshire""","""DepositAccounts""","""Accepted"""
"""UPlusFinPersonal""","""No Resp+""",533,"""Bergeville""",null,"""Rejected"""
"""BasicChecking""","""No Resp+""",555,"""Willyville""","""DepositAccounts""","""Rejected"""


As you can see, this dataset consists of regular predictors, IH predictors, context keys and the outcome column. Additionally, some columns are numeric, others are strings. Let's first initialize the DataAnonymization class.

In [4]:
anon = DataAnonymization(hds_folder='../../../../data/')

By default, the class applies a set of anonymisation techniques:
- Column names are remapped to a non-descriptive name
- Categorical values are hashed with a random seed
- Numerical values are normalized between 0 and 1
- Outcomes are mapped to a binary outcome.

To apply these techniques, simply call `.process()`:

In [5]:
anon.process()

PREDICTOR_0,filename,PREDICTOR_2,PREDICTOR_3,Context_Name,IH_PREDICTOR_0,Decision_Outcome
f64,str,str,str,str,str,bool
1.2167e18,"""../../../../data/SampleHDS.jso…","""6147846389658722553""","""2288875718529266449""","""4249663721192810638""","""10229436525961356759""",false
8.4508e18,"""../../../../data/SampleHDS.jso…","""9546562836500446922""","""1384965645898912072""","""4249663721192810638""","""17317120373498427018""",true
1.7675e19,"""../../../../data/SampleHDS.jso…","""1421966280809475974""","""17904252642100228159""","""2440746720503869809""","""10229436525961356759""",false
9.0321e18,"""../../../../data/SampleHDS.jso…","""9546562836500446922""","""13125856642762912113""","""14674378938643070016""","""10229436525961356759""",false
2.3087e18,"""../../../../data/SampleHDS.jso…","""6147846389658722553""","""4452067448264920949""","""14674378938643070016""","""15987834086564834453""",true
1.4344e19,"""../../../../data/SampleHDS.jso…","""1421966280809475974""","""11369660024816883368""","""15823670710176640254""",null,false
1.6626e19,"""../../../../data/SampleHDS.jso…","""1421966280809475974""","""16332345803428636889""","""14674378938643070016""","""15987834086564834453""",false


To trace back the columns to their original names, the class also contains a mapping, which does not have to be provided.

In [6]:
anon.column_mapping

{'Customer_CLV': 'PREDICTOR_0',
 'filename': 'filename',
 'Customer_MaritalStatus': 'PREDICTOR_2',
 'Customer_City': 'PREDICTOR_3',
 'Context_Name': 'Context_Name',
 'IH_Web_Inbound_Accepted_pxLastGroupID': 'IH_PREDICTOR_0',
 'Decision_Outcome': 'Decision_Outcome'}

## Configs

Each capability can optionally be turned off - see below for the full list of config options, and refer to the API reference for the full description.

In [7]:
dict(zip(Config.__init__.__code__.co_varnames[1:], Config.__init__.__defaults__))

{'config_file': None,
 'hds_folder': '.',
 'use_datamart': False,
 'datamart_folder': 'datamart',
 'output_format': 'ndjson',
 'output_folder': 'output',
 'mapping_file': 'mapping.map',
 'mask_predictor_names': True,
 'mask_context_key_names': False,
 'mask_ih_names': True,
 'mask_outcome_name': False,
 'mask_predictor_values': True,
 'mask_context_key_values': True,
 'mask_ih_values': True,
 'mask_outcome_values': True,
 'context_key_label': 'Context_*',
 'ih_label': 'IH_*',
 'outcome_column': 'Decision_Outcome',
 'positive_outcomes': ['Accepted', 'Clicked'],
 'negative_outcomes': ['Rejected', 'Impression'],
 'special_predictors': ['Decision_DecisionTime',
  'Decision_OutcomeTime',
  'Decision_Rank'],
 'sample_percentage_schema_inferencing': 0.01}

It's easy to change these parameters by just passing the keyword arguments. In the following example, we
- Keep the IH predictor names
- Keep the outcome values
- Keep the context key values
- Keep the context key predictor names

In [8]:
anon = DataAnonymization(
    hds_folder="../../../../data/",
    mask_ih_names=False,
    mask_outcome_values=False,
    mask_context_key_values=False,
    mask_context_key_names=False,
)
anon.process()


PREDICTOR_0,filename,PREDICTOR_2,PREDICTOR_3,Context_Name,IH_Web_Inbound_Accepted_pxLastGroupID,Decision_Outcome
f64,str,str,str,str,str,str
3.7144e18,"""../../../../data/SampleHDS.jso…","""5116113255984028597""","""9718179972840610916""","""FirstMortgage30yr""","""3904756066300791155""","""Rejected"""
1.7416e19,"""../../../../data/SampleHDS.jso…","""11206299812145901282""","""2606546173167921635""","""FirstMortgage30yr""","""2073517796186034520""","""Accepted"""
7.3266e18,"""../../../../data/SampleHDS.jso…","""18231480856777751062""","""15698682252462766581""","""MoneyMarketSavingsAccount""","""3904756066300791155""","""Rejected"""
4.8024e18,"""../../../../data/SampleHDS.jso…","""11206299812145901282""","""16203385141368777126""","""BasicChecking""","""3904756066300791155""","""Rejected"""
9.8712e18,"""../../../../data/SampleHDS.jso…","""5116113255984028597""","""6952337591127786287""","""BasicChecking""","""10099839356039493843""","""Accepted"""
3.7920e18,"""../../../../data/SampleHDS.jso…","""18231480856777751062""","""16041653561048533847""","""UPlusFinPersonal""",null,"""Rejected"""
1.3435e19,"""../../../../data/SampleHDS.jso…","""18231480856777751062""","""7277846800018900085""","""BasicChecking""","""10099839356039493843""","""Rejected"""


The configs can also be written and read as such:

In [9]:
anon.config.save_to_config_file('config.json')

In [10]:
anon = DataAnonymization(config=Config(config_file='config.json'))
anon.process()

PREDICTOR_0,filename,PREDICTOR_2,PREDICTOR_3,Context_Name,IH_Web_Inbound_Accepted_pxLastGroupID,Decision_Outcome
f64,str,str,str,str,str,str
1.2499e19,"""../../../../data/SampleHDS.jso…","""18293990344771377923""","""10401712909067898287""","""FirstMortgage30yr""","""7470891602762162258""","""Rejected"""
4.4825e18,"""../../../../data/SampleHDS.jso…","""6519604743693388226""","""17427030756996613094""","""FirstMortgage30yr""","""17303967803918605159""","""Accepted"""
1.7559e19,"""../../../../data/SampleHDS.jso…","""12013569918084110708""","""2116059138085243262""","""MoneyMarketSavingsAccount""","""7470891602762162258""","""Rejected"""
1.8016e18,"""../../../../data/SampleHDS.jso…","""6519604743693388226""","""16761861177852596977""","""BasicChecking""","""7470891602762162258""","""Rejected"""
1.6517e19,"""../../../../data/SampleHDS.jso…","""18293990344771377923""","""12818238716068238964""","""BasicChecking""","""4600961781277298053""","""Accepted"""
1.3564e19,"""../../../../data/SampleHDS.jso…","""12013569918084110708""","""3822978993054429388""","""UPlusFinPersonal""",null,"""Rejected"""
6.3226e17,"""../../../../data/SampleHDS.jso…","""12013569918084110708""","""12414514550878942568""","""BasicChecking""","""4600961781277298053""","""Rejected"""


## Exporting
Two functions export:
- `create_mapping_file()` writes the mapping file of the predictor names
- `write_to_output()` writes the processed dataframe to disk

Write to output accepts the following extensions: `["ndjson", "parquet", "arrow", "csv"]`

In [11]:
anon.create_mapping_file()
with open('mapping.map') as f:
    print(f.read())

Customer_CLV=PREDICTOR_0
filename=filename
Customer_MaritalStatus=PREDICTOR_2
Customer_City=PREDICTOR_3
Context_Name=Context_Name
IH_Web_Inbound_Accepted_pxLastGroupID=IH_Web_Inbound_Accepted_pxLastGroupID
Decision_Outcome=Decision_Outcome



In [12]:
anon.write_to_output(ext='arrow')

In [13]:
pl.read_ipc('output/hds.arrow')

PREDICTOR_0,PREDICTOR_2,PREDICTOR_3,Context_Name,IH_Web_Inbound_Accepted_pxLastGroupID,Decision_Outcome
f64,str,str,str,str,str
7.0882e17,"""16422462981718732615""","""7626264414049753285""","""FirstMortgage30yr""","""9571052652269174784""","""Rejected"""
1.6083e19,"""974077714294797116""","""11390115286477205611""","""FirstMortgage30yr""","""15926504249917894361""","""Accepted"""
7.7979e18,"""576385218094076406""","""18364511084597139830""","""MoneyMarketSavingsAccount""","""9571052652269174784""","""Rejected"""
1.6801e19,"""974077714294797116""","""13544611003525504311""","""BasicChecking""","""9571052652269174784""","""Rejected"""
4.6573e17,"""16422462981718732615""","""2847659491701829113""","""BasicChecking""","""4292160008144934078""","""Accepted"""
1.7649e19,"""576385218094076406""","""6698486850386849814""","""UPlusFinPersonal""",null,"""Rejected"""
1.2700e19,"""576385218094076406""","""12350877057713676354""","""BasicChecking""","""4292160008144934078""","""Rejected"""


## Advanced: Hash fuctions

By default, we use [the same hashing algorithm Polars](https://pola-rs.github.io/polars/py-polars/html/reference/expressions/api/polars.Expr.hash.html#polars.Expr.hash) uses: [xxhash](https://github.com/Cyan4973/xxHash), as implemented [here](https://github.com/pola-rs/polars/blob/3f287f370b3c388ed2f3f218b2c096382548136f/polars/polars-core/src/vector_hasher.rs#L266). xxhash is fast to compute, and you can check its performance in collision, dispersion and randomness [here](https://github.com/Cyan4973/xxHash/tree/dev/tests). 

xxhash accepts four distinct seeds, but by default we set the seeds to `0`. It is possible to set the `seed` argument of the `process()` function to `'random'`, which will set all four seeds to a random integer between `0` and `1000000000`. Alternatively, it is possible to supply the four seeds manually with arguments `seed`, `seed_1`, `seed_2` and `seed_3`. 

If the xxhash with (random) seed(s) is not deemed sufficiently secure, it is possible to use your own hashing algorithm.

Note that since we're now running python code and not native Polars code anymore, this will be _significantly_ slower. Nonetheless, it is possible.

Just as an example - this is how one would use sha3_256:

In [14]:
from hashlib import sha3_256

anon.process(algorithm=lambda x: sha3_256(x.encode()).hexdigest())

sys:1: MapWithoutReturnDtypeWarning:

Calling `map_elements` without specifying `return_dtype` can lead to unpredictable results. Specify `return_dtype` to silence this warning.

sys:1: MapWithoutReturnDtypeWarning:

Calling `map_elements` without specifying `return_dtype` can lead to unpredictable results. Specify `return_dtype` to silence this warning.

sys:1: MapWithoutReturnDtypeWarning:

Calling `map_elements` without specifying `return_dtype` can lead to unpredictable results. Specify `return_dtype` to silence this warning.

sys:1: MapWithoutReturnDtypeWarning:

Calling `map_elements` without specifying `return_dtype` can lead to unpredictable results. Specify `return_dtype` to silence this warning.



ComputeError: AttributeError: 'int' object has no attribute 'encode'